In [1]:
#가상환경 py310_2
#########
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import math

In [2]:
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import correlate
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import random
from datetime import datetime, timedelta
from collections import Counter
#라벨링 툴 
con=pymysql.connect(host='192.168.0.68',port=3306,user='FS_MNG2', password='7HkY3pAq9', db='SCHOOLSAFE_CLASSIFICATION', charset='utf8') 
cur = con.cursor()
#sql = "SELECT SNSRID FROM wj_test_f_sensor_data2023 LIMIT 50" 
#################################################전력 사용량###################################################


Duplicate key in file '/home/fs-ai/.config/matplotlib/matplotlibrc', line 3 ('font.family : NanumGothic')
Duplicate key in file '/home/fs-ai/.config/matplotlib/matplotlibrc', line 4 ('font.size : 15.0')
2023-09-15 13:27:05.673174: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 13:27:05.743554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 13:27:06.700117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
with con:
    cur = con.cursor()
    
    # Corrected SQL statement
    sql = "SELECT ACC_ID, USR_ID, ACC_PLACE, ACC_TIME, ACC_ACTIVITY, ACC_MEDIATOR, ACC_TYPE, ACC_INJURY, ACC_SUBJECTIVE FROM C_ACCIDENT_CLASSIFICATION"
    
    cur.execute(sql)
    rows = cur.fetchall()
    print(rows)

# Convert to DataFrame
df1_1 = pd.DataFrame(rows)

((1, 4, None, None, 242, 282, 337, None, None), (2, 4, None, None, 212, 329, 337, None, 129), (3, 4, None, 183, 239, 323, 337, None, 129), (4, 4, None, None, None, 301, None, None, 393), (5, 4, None, None, 218, 303, None, None, 129), (6, 4, None, None, 258, 330, 337, None, None), (7, 4, None, 190, 264, 309, 338, None, 129), (8, 4, 174, None, None, 327, None, None, 129), (9, 4, 177, None, None, 275, None, None, 129), (10, 4, 156, None, None, 279, None, None, 129), (11, 3, 177, None, None, 309, None, None, 129), (12, 3, 161, 199, None, 329, 337, None, 128), (13, 3, 160, None, None, 330, 337, None, 129), (14, 3, 178, None, 267, 330, None, None, 393), (15, 3, 161, None, 238, 309, None, None, 129), (16, 3, 177, None, 263, 293, None, None, 129), (17, 3, 172, None, 253, 303, 331, None, 129), (18, 3, 177, None, None, 324, None, None, 129), (19, 3, 174, None, None, 330, None, None, 129), (20, 3, 161, None, 238, 309, None, None, 129), (21, 4, None, None, None, 309, None, None, 129), (22, 4, None

In [4]:
df1_1

,0,1,2,3,4,5,6,7,8
0,1,4,NaN,NaN,242.0,282.0,337.0,None,NaN
1,2,4,NaN,NaN,212.0,329.0,337.0,None,129.0
2,3,4,NaN,183.0,239.0,323.0,337.0,None,129.0
3,4,4,NaN,NaN,NaN,301.0,NaN,None,393.0
4,5,4,NaN,NaN,218.0,303.0,NaN,None,129.0
...,...,...,...,...,...,...,...,...,...
1906,2626,4,NaN,NaN,NaN,279.0,NaN,None,NaN
1907,2627,4,NaN,NaN,NaN,323.0,NaN,None,NaN
1908,2628,4,NaN,NaN,NaN,277.0,337.0,None,NaN
1909,2629,4,NaN,190.0,NaN,289.0,337.0,None,NaN


In [5]:
df1_2=df1_1.rename(columns={0:"id", 1:"USR_ID", 2:"신분류_사고장소", 3:"신분류_사고시간", 4:"신분류_사고활동",5:"신분류_매개물",6:"신분류_사고형태",7:"신분류_사고부위", 8:"신분류_주관적요인"})

In [7]:
df1_2

,id,USR_ID,신분류_사고장소,신분류_사고시간,신분류_사고활동,신분류_매개물,신분류_사고형태,신분류_사고부위,신분류_주관적요인
0,1,4,NaN,NaN,242.0,282.0,337.0,None,NaN
1,2,4,NaN,NaN,212.0,329.0,337.0,None,129.0
2,3,4,NaN,183.0,239.0,323.0,337.0,None,129.0
3,4,4,NaN,NaN,NaN,301.0,NaN,None,393.0
4,5,4,NaN,NaN,218.0,303.0,NaN,None,129.0
...,...,...,...,...,...,...,...,...,...
1906,2626,4,NaN,NaN,NaN,279.0,NaN,None,NaN
1907,2627,4,NaN,NaN,NaN,323.0,NaN,None,NaN
1908,2628,4,NaN,NaN,NaN,277.0,337.0,None,NaN
1909,2629,4,NaN,190.0,NaN,289.0,337.0,None,NaN


In [9]:
import pymysql

try:
    con = pymysql.connect(host='192.168.0.68', port=3306, user='FS_MNG2', password='7HkY3pAq9', db='SCHOOLSAFE_CLASSIFICATION', charset='utf8')
    cur2 = con.cursor()

    # 올바른 SQL 문장
    sql2 = "SELECT CLAS_ID, CLAS_LEVEL, CLAS_NAME, CLAS_TYPE, CLAS_SUB_COUNT, CLAS_HIGH_ID FROM C_CLASSIFICATION"

    cur2.execute(sql2)
    rows2 = cur2.fetchall()
    print(rows2)
    
    # 데이터베이스 커넥션 닫기
    con.close()
except pymysql.Error as e:
    print(f"오류 발생: {e}")


((1, 1, '사고장소', '', None, None), (2, 1, '사고시간', '', None, None), (3, 1, '사고당시활동', '', None, None), (4, 1, '매개물', '', None, None), (5, 1, '사고형태', '', None, None), (6, 1, '사고부위', '', None, None), (7, 1, '주관적요인', '', None, None), (8, 2, '교내', '', 23, 1), (9, 2, '교외', '', 25, 1), (10, 2, '교육과정-교과(초·중·고)', '', 5, 2), (11, 2, '교육과정(유치원)', '', 8, 2), (12, 2, '교육과정-비교과(공통)', '', 10, 2), (13, 2, '기타 교육활동시간(공통)', '', 10, 2), (14, 2, '학습활동', '', 5, 3), (15, 2, '스포츠활동', '', 45, 3), (16, 2, '일상활동', '', 6, 3), (17, 2, '이동', '', 2, 3), (18, 2, '기타', '', 2, 3), (19, 2, '건축물', '', 11, 4), (20, 2, '교통수단', '', 6, 4), (21, 2, '기구(도구,기계 등)', '', 34, 4), (22, 2, '사람', '', 1, 4), (23, 2, '기타', '', 6, 4), (24, 2, '없음', '', 1, 4), (25, 2, '둔탁한 힘에 의한 사고', '', 12, 5), (26, 2, '관통/침투 하는 힘에 의한 사고', '', 4, 5), (27, 2, '열기(전기 포함) 또는 냉기에 의한 사고', '', 5, 5), (28, 2, '기타 손상 사고', '', 8, 5), (29, 2, '머리와 목', '', 12, 6), (30, 2, '몸통', '', 5, 6), (31, 2, '사지', '', 14, 6), (32, 2, '기타', '', 1, 6), (33, 2, '비의도', '', 4, 7), (

In [10]:
cls1 = pd.DataFrame(rows2)

In [11]:
cls1

,0,1,2,3,4,5
0,1,1,사고장소,,NaN,NaN
1,2,1,사고시간,,NaN,NaN
2,3,1,사고당시활동,,NaN,NaN
3,4,1,매개물,,NaN,NaN
4,5,1,사고형태,,NaN,NaN
...,...,...,...,...,...,...
388,389,4,발,left_right,0.0,124.0
389,390,4,발가락,left_right,0.0,124.0
390,391,4,기타(직접 기입식),checkbox,0.0,125.0
391,392,2,해당없음,,1.0,7.0


In [12]:
cls1=cls1.rename(columns={0:"CLAS_ID", 1:"CLAS_LEVEL", 2:"CLAS_NAME", 3:"CLAS_TYPE", 4:"CLAS_SUB_COUNT",5:"CLAS_HIGH_ID"})

In [13]:
cls1

,CLAS_ID,CLAS_LEVEL,CLAS_NAME,CLAS_TYPE,CLAS_SUB_COUNT,CLAS_HIGH_ID
0,1,1,사고장소,,NaN,NaN
1,2,1,사고시간,,NaN,NaN
2,3,1,사고당시활동,,NaN,NaN
3,4,1,매개물,,NaN,NaN
4,5,1,사고형태,,NaN,NaN
...,...,...,...,...,...,...
388,389,4,발,left_right,0.0,124.0
389,390,4,발가락,left_right,0.0,124.0
390,391,4,기타(직접 기입식),checkbox,0.0,125.0
391,392,2,해당없음,,1.0,7.0


In [30]:
merged_df = df1_2.copy()

In [31]:
merged_df = merged_df.merge(cls1[['CLAS_ID','CLAS_NAME']], how='left', left_on='신분류_사고장소', right_on='CLAS_ID')

In [32]:
merged_df

,id,USR_ID,신분류_사고장소,신분류_사고시간,신분류_사고활동,신분류_매개물,신분류_사고형태,신분류_사고부위,신분류_주관적요인,CLAS_ID,CLAS_NAME
0,1,4,NaN,NaN,242.0,282.0,337.0,None,NaN,NaN,NaN
1,2,4,NaN,NaN,212.0,329.0,337.0,None,129.0,NaN,NaN
2,3,4,NaN,183.0,239.0,323.0,337.0,None,129.0,NaN,NaN
3,4,4,NaN,NaN,NaN,301.0,NaN,None,393.0,NaN,NaN
4,5,4,NaN,NaN,218.0,303.0,NaN,None,129.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1906,2626,4,NaN,NaN,NaN,279.0,NaN,None,NaN,NaN,NaN
1907,2627,4,NaN,NaN,NaN,323.0,NaN,None,NaN,NaN,NaN
1908,2628,4,NaN,NaN,NaN,277.0,337.0,None,NaN,NaN,NaN
1909,2629,4,NaN,190.0,NaN,289.0,337.0,None,NaN,NaN,NaN


In [ ]:
#구분류 
conditions = [
    df1_2['신분류_사고장소'] == '겨드랑이',#1
    df9['신분류_사고장소'] == '견갑골',#2
    df9['사고부위'] == '기타',#3
    df9['사고부위'] == '내장기관',#4
    df9['사고부위'] == '넓적다리(허벅지)',#5
    df9['사고부위'] == '눈',#6
    df9['사고부위'] == '대퇴골',#7
    df9['사고부위'] == '두피',#8
    df9['사고부위'] == '등',#9
    df9['사고부위'] == '목',#10
    df9['사고부위'] == '무릎',#11
    df9['사고부위'] == '발',#12
    df9['사고부위'] == '발가락',#13
    df9['사고부위'] == '발목',#14
    df9['사고부위'] == '배',#15
    df9['사고부위'] == '볼',#16
    df9['사고부위'] == '불상',#17
    df9['사고부위'] == '손',#18
    df9['사고부위'] == '손가락',#19
    df9['사고부위'] == '손목',#20
    df9['사고부위'] == '쇄골',#21
    df9['사고부위'] == '신경계',#22
    df9['사고부위'] == '아래다리',#23
    df9['사고부위'] == '아래팔',#24
    df9['사고부위'] == '어깨',#25
    df9['사고부위'] == '엉덩이',#26
    df9['사고부위'] == '위팔',#27
    df9['사고부위'] == '이마',#28
    df9['사고부위'] == '입(구강/혀)',#29
    df9['사고부위'] == '종아리',#30
    df9['사고부위'] == '치아',#31
    df9['사고부위'] == '코',#32
    df9['사고부위'] == '턱',#33
    df9['사고부위'] == '팔꿈치',#34
    df9['사고부위'] == '허리',#35
    df9['사고부위'] == '흉부(가슴)',#36
    ]


#신분류
values = ['몸통-흉복부-흉부',#1
        '몸통-등‧허리-등',#2
        '기타-기타-기타',#3
        '몸통-흉복부-내장기관',#4
        '사지-다리-넓적다리(허벅지)',#5
        '머리와 목-머리(입 제외)-눈',#6
        '사지-다리-넓적다리(허벅지)',#7
        '머리와 목-머리(입 제외)-두피',#8
        '몸통-등‧허리-등',#9
        '머리와 목-목-목',#10
        '사지-다리-무릎',#11
        '사지-발목 및 발가락-발',#12
        '사지-발목 및 발가락-발가락',#13
        '사지-발목 및 발가락-발목',#14
        '몸통-흉복부-복부',#15
        '머리와 목-머리(입 제외)-볼',#16
        '기타-기타-기타',#17
        '사지-손목 및 손-손',#18
        '사지-손목 및 손-손가락',#19
        '사지-손목 및 손-손목',#20
        '사지-팔-위팔',#21 #쇄골
        '기타-기타-기타',#22
        '사지-다리-아래다리(종아리)',#23
        '사지-팔-아래팔',#24
        '사지-팔-어깨',#25
        '사지-다리-골반/엉덩이',#26
        '사지-팔-위팔',#27
        '머리와 목-머리(입 제외)-이마',#28
        '머리와 목-입-입술 및 구강',#29
        '사지-다리-아래다리(종아리)',#30
        '머리와 목-입-치아',#31
        '머리와 목-머리(입 제외)-코',#32
        '머리와 목-머리(입 제외)-턱',#33
        '사지-팔-팔꿈치',#34
        '몸통-등‧허리-허리',#35
        '몸통-흉복부-흉부'#36
        ]

df1_3['신분류_사고부위2'] = np.select(conditions, values, default=np.NaN)

In [ ]:
#indong.rename(columns={"0":"id", "1":"time", "2":"igr"})
#indong.columns=['id','time', 'SNSRKWH']
#indong['id2'] = (indong.groupby('id').ngroup() + 1).astype(str)
#indong.sort_values(by=['id2', 'time'], inplace=True)
#indong2=indong
#indong2
#indong2['time'] = pd.to_datetime(indong2['time'])
#indong2['id2'] =pd.to_numeric(indong2['id2'])
#unique = indong2['id2'].unique()

In [19]:
df11=pd.read_csv("/home/fs-ai/Desktop/Woojin_Kim/NLP_text classification/School/clean_data_V0.1/all_df11_total.csv")

/tmp/ipykernel_33314/3916188186.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df11=pd.read_csv("/home/fs-ai/Desktop/Woojin_Kim/NLP_text classification/School/clean_data_V0.1/all_df11_total.csv")
